In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
url1 = 'https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-cities-time.csv'
url2 = 'https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv'
municipiosTodos = pd.read_csv(url1)
estados = pd.read_csv(url2)

In [3]:
display(municipiosTodos)
display(estados)

,date,country,state,city,ibgeID,newDeaths,deaths,newCases,totalCases
0,2020-02-25,Brazil,SP,São Paulo/SP,3550308,0,0,1,1
1,2020-02-25,Brazil,TOTAL,TOTAL,0,0,0,1,1
2,2020-02-26,Brazil,SP,São Paulo/SP,3550308,0,0,0,1
3,2020-02-26,Brazil,TOTAL,TOTAL,0,0,0,0,1
4,2020-02-27,Brazil,SP,São Paulo/SP,3550308,0,0,0,1
...,...,...,...,...,...,...,...,...,...
14435,2020-04-14,Brazil,GO,Águas Lindas de Goiás/GO,5200258,0,0,0,2
14436,2020-04-14,Brazil,SC,Águas Mornas/SC,4200606,0,0,0,1
14437,2020-04-14,Brazil,SP,Águas de Lindóia/SP,3500501,0,1,0,1
14438,2020-04-14,Brazil,SP,Águas de São Pedro/SP,3500600,0,0,0,1


,date,country,state,city,newDeaths,deaths,newCases,totalCases,deathsMS,totalCasesMS
0,2020-02-25,Brazil,SP,TOTAL,0,0,1,1,0,0
1,2020-02-25,Brazil,TOTAL,TOTAL,0,0,1,1,0,0
2,2020-02-26,Brazil,SP,TOTAL,0,0,0,1,0,1
3,2020-02-26,Brazil,TOTAL,TOTAL,0,0,0,1,0,1
4,2020-02-27,Brazil,SP,TOTAL,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...
933,2020-04-14,Brazil,SC,TOTAL,0,26,27,853,26,826
934,2020-04-14,Brazil,SE,TOTAL,0,4,0,45,4,45
935,2020-04-14,Brazil,SP,TOTAL,87,695,476,9371,695,9371
936,2020-04-14,Brazil,TO,TOTAL,0,0,0,26,0,26


### Após carregar os 2 datasets, um contém apenas a estatística por Estados Brasileiros e o outro possui as estatísticas mais detalhadas sobre os muncípios, então, resolvo cruzar as informações com a finalidade de comparar o caso total de COVID19 por um Estado, sua capital e alguma outra cidade referente a este Estado.

Crio um dataframe com todos os muncípios com casos de Minas Gerais que foram cadastrados

In [ ]:
estado_mg = estados[estados.state.str.contains('MG')]
datamg = municipiosTodos[municipiosTodos.city.str.contains('/MG')]
datamg.city = [x.rstrip('/MG') for x in datamg.city]
datamg.city = [x.replace('TOTAL','MG') for x in datamg.city]
display(datamg)

In [ ]:
#salvo em csv e json para outras análises
datamg.to_csv('mg.csv', index='index')
datamg.to_json('mg.json',orient='index')

## Resolvo comparar uma cidade de Minas com sua sua capital e todos os casos do Estado

In [ ]:
comparar = datamg[datamg.city.str.contains('Juiz de Fora|Belo Horizonte')]
estado_mg.set_index('date',inplace=True)
comparar.set_index('date',inplace=True)
comparar = pd.concat([estado_mg, comparar], join="inner")
comparar.drop(['country', 'state','newDeaths', 'deaths', 'newCases'],axis=1,inplace=True)
comparar.reset_index(inplace=True)
comparar

In [ ]:
comparar.date = pd.to_datetime(comparar.date)
fig, ax = plt.subplots(figsize=(16,9))
sns.set()
ax = sns.lineplot(x='date', y='totalCases', hue='city',data=comparar, markers=True, dashes=False, palette="Reds", legend="full")
plt.xlabel('data', size=20)
plt.ylabel('casos', size=20)
plt.title('Número de casos por dia', size=20)
plt.yticks(np.arange(0, max(comparar.totalCases)+100, 30))
plt.tight_layout()
plt.savefig('jfbh.png')

## Outra forma de ver a progressão é olhar para um mapa de calor separado em período de 3 dias e assim, temos uma outra forma de ver a intensidade da progressão dos casos

In [ ]:
# crio um dataframe com as datas de 3 em 3 dias
date = pd.date_range(comparar.date[0], periods=3)

# cio uma tabela pivot para comparar o heatmap e ver a relação do COVID-19 com as datas
piv = pd.pivot_table(comparar, values='totalCases',index=comparar.date.dt.strftime('%m-%d'), columns=comparar.city, fill_value=0)
fig, ax = plt.subplots(figsize=(16,9))
sns.heatmap(piv, cmap='Reds')